<a href="https://colab.research.google.com/github/lfresard/toolbox/blob/master/Candidate_Gene_Prioritization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gene prioritization

A hands-on rare disease diagnostic workshop on RNA-seq data @ ASHG19

---





## Setup the notebook

To get started we have to setup this IR notebook @ Colab. To do so please run the following code snippet.
This will download all nessesary software packages for this tutorial. Also it will download the example data sets used throughout this tutorial.


In [1]:
download.file("https://raw.githubusercontent.com/c-mertes/RNAseq-ASHG19/master/r-env-setup-script.R", "r-env-setup-script.R")
source("r-env-setup-script.R")

Update and install needed Ubuntu packages
Download R package cache
Unzipping R package cache
Retrieve data for tutorials


## Set up working session
---

First a few packages need to be loaded. \\
(1) `VariantAnnotation`: annotate variants, compute amino acid coding changes, predict coding outcomes. \\
(2) `TVTB`: filter, summarise and visualise genetic variation data stored in VCF files. \\
(3) `annotables`: annotating/converting Gene IDs. \\
(4) `tidyverse`: collection of packages designed for data science \\
(5) `data.table`: data manipulation operations 

In [2]:
library(VariantAnnotation)
library(TVTB)
library(annotables)
library(tidyverse)
library(data.table)


Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: GenomeInfoDb
Loading required package: S4Vectors
Loading required package: stats4

Attac


**It is not uncommon to have more than one candidate outlier gene when looking at expression, splicing or allele specific events. It is useful to kown that there are ways to prioritize those genes.
Here we will see how coming back to phenotype and genotype data can help reduce the list of candidates in a case informed way.** 


## Case 1
This boy was the third child of healthy non-consanguineous French parents. Pregnancy and delivery were uneventful. Early psychomotor development was normal. However, speech development was delayed, acquiring language at the age of 4. At 11, he began to experience psychomotor regression and progressive visual loss. At current (by the time of the publication) age of 47, he has severe walking difficulties, blindness, abnormal behaviour (easily frightened, sometimes aggressive) and spontaneous speech.

In our dataset, this sample is named **NA11918**

### Get list of candidate genes from OUTRIDER
Here we download the list of outlier gene that you generated using OUTRIDER in an earlier session.

In [4]:
# Download list of candidates
outrider_results_genes=fread("https://i12g-gagneurweb.in.tum.de/public/workshops/RNAseq_ASHG19/input_data/outrider/results_pvalue.tsv.gz", drop=1)

# Filter for the case of interest
outrider_results_genes=outrider_results_genes%>%filter(sampleID=="NA11918") %>% select(geneID) %>% unlist

# Take a look at what the genes list look like
print("This is what the output looks like")
head(outrider_results_genes)

# Get the number of candidate genes for that case
print("In total there are this many candidate genes for this case")
length(outrider_results_genes)


Warning message in fread("https://i12g-gagneurweb.in.tum.de/public/workshops/RNAseq_ASHG19/input_data/outrider/results_pvalue.tsv.gz", :
"Detected 14 column names but the data has 15 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file."

[1] "This is what the output looks like"


geneID1                geneID2                geneID3 
"ENSG00000165288.10_2" "ENSG00000179715.12_4" "ENSG00000146021.14_4" 
               geneID4                geneID5                geneID6 
"ENSG00000047932.13_4"  "ENSG00000115825.9_2" "ENSG00000119772.16_3"

[1] "In total there are this many candidate genes for this case"


[1] 22

You can see that the gene names are not totally in an Ensembl format. The gene names need to be reformated so that we keep only the official gene name in our analyses.

In [5]:
# Modify gene format
outrider_results_genes=str_extract(outrider_results_genes, "ENSG[0-9]+")


# Look at the changes
print("This is what the output looks like")
head(outrider_results_genes)


[1] "This is what the output looks like"


[1] "ENSG00000165288" "ENSG00000179715" "ENSG00000146021" "ENSG00000047932"
[5] "ENSG00000115825" "ENSG00000119772"

Now the gene name correspond to the official Ensembl symbols. It is possible to get a bit more information on those. 
In particular we can easily obtain their symbol and description using the package ` annotable `.


In [6]:
# Query annotable Grch37 object and keep only genes that are in the list of outliers
candidate_genes=grch37 %>% filter(ensgene %in% outrider_results_genes)

# Look at the results
print("This is what the output looks like")
head(candidate_genes)

[1] "This is what the output looks like"


ensgene,entrez,symbol,chr,start,end,strand,biotype,description
<chr>,<int>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>
ENSG00000047932,57120,GOPC,6,117639374,117923691,-1,protein_coding,golgi-associated PDZ and coiled-coil motif containing [Source:HGNC Symbol;Acc:17643]
ENSG00000074706,26034,IPCEF1,6,154475631,154677926,-1,protein_coding,interaction protein for cytohesin exchange factors 1 [Source:HGNC Symbol;Acc:21204]
ENSG00000090674,57192,MCOLN1,19,7587512,7598895,1,protein_coding,mucolipin 1 [Source:HGNC Symbol;Acc:13356]
ENSG00000091490,23231,SEL1L3,4,25749055,25865382,-1,protein_coding,sel-1 suppressor of lin-12-like 3 (C. elegans) [Source:HGNC Symbol;Acc:29108]
ENSG00000101096,4773,NFATC2,20,50003494,50179370,-1,protein_coding,"nuclear factor of activated T-cells, cytoplasmic, calcineurin-dependent 2 [Source:HGNC Symbol;Acc:7776]"
ENSG00000115825,23683,PRKD3,2,37477645,37551951,-1,protein_coding,protein kinase D3 [Source:HGNC Symbol;Acc:9408]


We now have the list of candidate genes and a bit more information on what they are. We are ready to filter this list to keep only the genes that are linked to the symptoms of the case of interest.

In our case, symptoms are:
* Developmental regression (HP:0002376)
* Ataxia(HP:0001251)
* Ophthalmoplegia (HP:0000602)
* Visual impairment (HP:0000505)
As you can see, there is a code beside each symptom. Those are HPO terms IDs, as found in the Human Phenotype Ontology database (https://hpo.jax.org/app/).


### Annotate candidate genes with HPO terms
#### Read in sample phenotype metadata


In [0]:
# We define the list of HPO terms IDs for the case
sample_HPO=c("HP:0002376","HP:0001251", "HP:0000602","HP:0000505")


#### Load gene to HPO ID file from HPO database 
The Human Phenotype Ontology gives access to a very useful file, showing the link between symptoms and genes. This file is updated regularly so we recommend to download the latest version whenever possible.

**bold text**
You can download this file here: http://compbio.charite.de/jenkins/job/hpo.annotations.monthly/lastStableBuild/artifact/annotation/ALL_SOURCES_FREQUENT_FEATURES_phenotype_to_genes.txt

In [10]:

# Get the file URL
hpo_annotations_url ="http://compbio.charite.de/jenkins/job/hpo.annotations.monthly/lastStableBuild/artifact/annotation/ALL_SOURCES_FREQUENT_FEATURES_phenotype_to_genes.txt"

# Extract the file name 
file <- basename(hpo_annotations_url)

# Read the file
gene_hpo <-fread(hpo_annotations_url, skip=1)

# Select only the columns of interest (Gene and HPO term)
gene_hpo <- gene_hpo[, c(4, 1)]
colnames(gene_hpo) <- c("Gene", "Term")

# Look at the results
print("This is what the output looks like")
head(gene_hpo)

[1] "This is what the output looks like"


Gene,Term
<chr>,<chr>
GLI3,HP:0001459
LMBR1,HP:0006088
SHH,HP:0010708
LMBR1,HP:0010708
GLI3,HP:0010713
PRPS1,HP:0030927


#### Get subset of genes that could match symptoms list
Now that we have downloaded that file, we want to keep only the genes that are phenotypically relevant to the case.

1. Filter for HPO terms corresponding to the case
2. Obtain the Ensembl IDs for the genes that are associated to those HPO terms


In [0]:
# Get list of genes corresponding to the case symptoms.
genes_hpo_case=gene_hpo %>% filter(Term %in% sample_HPO) %>% left_join(grch37,by=c("Gene"="symbol"))%>% select(ensgene) %>% unique%>% unlist


`genes_hpo_case` contains the Ensembl IDs of genes associated with HPO terms for the case.

In [13]:
print("This is what the output looks like")
head(genes_hpo_case)

[1] "This is what the output looks like"


ensgene1          ensgene2          ensgene3          ensgene4 
"ENSG00000187566" "ENSG00000171723" "ENSG00000105695" "ENSG00000136463" 
         ensgene5          ensgene6 
"ENSG00000081307" "ENSG00000182372"

You can quicly check how many genes are currently linked with the symptoms of your case of interest.

In [14]:
# Get number of genes linked to case symptoms
length(genes_hpo_case)

[1] 1519

There are 1,519 genes that could match some of the patient's symtoms.



---



#### Subset the list of genes to outlier candidates 
Here the assumption is that if there is an expression perturbation on the causal gene, this gene is somehow linked to some of the symtoms of the patient.\
So we want to **filter** the list of genes somehow linked to the patients symptoms (listed in `genes_hpo_case`) for the one obtained as candidates in previous work.


In [0]:
# Filter outlier genes for genes linked to the phenotype.
candidate_genes.hpo=candidate_genes %>% filter(ensgene %in% genes_hpo_case)

As you can see the list of potential genes is now much reduced.

In [17]:
# Get the number of genes left
print ("This is the number of outlier genes left:")
length(candidate_genes.hpo$ensgene)

# Take a look at the results
print("Let's take a look at those genes")
candidate_genes.hpo

[1] "This is the number of outlier genes left:"


[1] 2

[1] "Let's take a look at those genes"


ensgene,entrez,symbol,chr,start,end,strand,biotype,description
<chr>,<int>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>
ENSG00000090674,57192,MCOLN1,19,7587512,7598895,1,protein_coding,mucolipin 1 [Source:HGNC Symbol;Acc:13356]
ENSG00000119772,1788,DNMT3A,2,25455845,25565459,-1,protein_coding,DNA (cytosine-5-)-methyltransferase 3 alpha [Source:HGNC Symbol;Acc:2978]


We dropped from 22 candidates to 2 genes, *MCOLN1* and *DNMT3A*.




---

---



### Annotate with variant information

We can go further and annotate our candidate with the variant information for the case.

To do so, we need to transform the candidate genes list into the right format so that we can use the genes when reading in the vcf file in R.

#### Build a GRanges object with list of genes obtain in previous steps
We are using R packages that have been developped to filter vcf files. 
Here we want to upload only the part of the vcf file that is in regions of interest (ie our candidate genes). We can do this by giving as an input an object of type GRange (https://web.mit.edu/~r/current/arch/i386_linux26/lib/R/library/GenomicRanges/html/GRanges-class.html).


The next steps are transforming our list of candidate genes to the right format so that we can proceed with the analysis.

1. Change start and end position to include potential regulatory regions (+/- 1kb here).
note that you can customize this distance or use the gene coordinates only.


In [0]:
# Add new start and stop coordinates for genes of interest extending to 1kb around the gene
candidate_genes=candidate_genes.hpo %>% mutate(new_start=pmin(start,end)-1e3, new_end=pmax(start,end)+1e3)


# Take a look at the results
print("Let's take a look at those genes")
head(candidate_genes)

ensgene,entrez,symbol,chr,start,end,strand,biotype,description,new_start,new_end
<chr>,<int>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>
ENSG00000090674,57192,MCOLN1,19,7587512,7598895,1,protein_coding,mucolipin 1 [Source:HGNC Symbol;Acc:13356],6587512,8598895
ENSG00000119772,1788,DNMT3A,2,25455845,25565459,-1,protein_coding,DNA (cytosine-5-)-methyltransferase 3 alpha [Source:HGNC Symbol;Acc:2978],24455845,26565459


2. Transform data frame into GRange object

In [0]:
# Create a GRange Object
candidate_genes.gr=makeGRangesFromDataFrame(candidate_genes,ignore.strand=T,start.field="new_start",end.field="new_end")

3. Add gene names in GRanges object


In [0]:
# Add the Ensembl gene names to the GRange object
names(candidate_genes.gr)=candidate_genes$ensgene

#### Download the vcf file
Now that we have the coordinates of the candidate genes in the right format, we can load the vcf.

In [0]:
# download the vcf file
vcfRemote <- "https://i12g-gagneurweb.in.tum.de/public/workshops/RNAseq_ASHG19/input_data/variants/1000G_subset_exome.vep.vcf.gz"
system(paste0("wget -c ", vcfRemote), intern=TRUE)

# index the vcf file
indexVcf("1000G_subset_exome.vep.vcf.gz")

# create a reference 
vcfFile <- TabixFile("1000G_subset_exome.vep.vcf.gz")






character(0)

class: VcfFile 
path: 1000G_subset_exome.vep.vcf.gz
index: 1000G_subset_exome.vep.vcf.gz.tbi
isOpen: FALSE 
yieldSize: NA 


#### Create a parameter object influencing which sample is imported from the VCF file
Here only the sample of interest and variants overlapping the genes of interest will be looked at.


In [0]:
params=ScanVcfParam(samples="NA11918", which=candidate_genes.gr)


#### Read in vcf file filtering for sample of interest

In [0]:
vcf_rng <- readVcf(vcfFile, "hg19", params)



Warning message in .bcfHeaderAsSimpleList(header):
"duplicate keys in header will be forced to unique rownames"

#### Filter for Heterozygous or Homozygous Alt and genes of interest

In [0]:
# Filter on variants het or homozygous alt
Hetfilt <- FilterRules(list(HetorHomAlt = function(x) geno(x)$GT %in% c("0|1", "1|1")))
GeneFilt<-VcfVepRules(exprs = list(Cand_genes = expression(SYMBOL %in% c("MCOLN1", "DNMT3A") )))                            

combinedPreFilters <- VcfFilterRules(
  Hetfilt, 
  GeneFilt)
vcf_het_cand <- subsetByFilter(vcf_rng, combinedPreFilters)


#### Get consequence field


In [0]:
csq <- ensemblVEP::parseCSQToGRanges(x = vcf_het_cand)
head(csq)
csq$SYMBOL

GRanges object with 6 ranges and 58 metadata columns:
              seqnames    ranges strand |      Allele
                 <Rle> <IRanges>  <Rle> | <character>
  rs768736321       19   7592731      * |           C
  rs768736321       19   7592731      * |           C
  rs768736321       19   7592731      * |           C
  rs768736321       19   7592731      * |           C
  rs768736321       19   7592731      * |           C
  rs768736321       19   7592731      * |           C
                                     Consequence      IMPACT      SYMBOL
                                     <character> <character> <character>
  rs768736321                     intron_variant    MODIFIER      MCOLN1
  rs768736321 non_coding_transcript_exon_variant    MODIFIER      MCOLN1
  rs768736321              upstream_gene_variant    MODIFIER      MCOLN1
  rs768736321              upstream_gene_variant    MODIFIER      MCOLN1
  rs768736321            downstream_gene_variant    MODIFIER      MCOLN1
  r

[1] "MCOLN1" "MCOLN1" "MCOLN1" "MCOLN1" "MCOLN1" "MCOLN1" "MCOLN1" "MCOLN1"
 [9] "MCOLN1" "MCOLN1" "MCOLN1" "MCOLN1" "MCOLN1" "MCOLN1" "MCOLN1" "MCOLN1"
[17] "MCOLN1" "MCOLN1" "MCOLN1" "MCOLN1"



#### Define a set of filters



In [0]:

# Filter on distance to the gene
vepDistFilter <- VcfVepRules(exprs = list(Distance = expression(DISTANCE <=1000)))

# Filter on allele frequency
vepMAFFilter<- VcfVepRules(exprs = list(MAF = expression(as.numeric(gnomAD_AF) <=0.01)))


# Filter on CADD score
vepCADDFilter <- VcfVepRules(exprs = list(CADD = expression(CADD_PHRED >=20)))


# Filter on consequences

highImpactVariant<-VcfVepRules(exprs = list(BigImpact = expression(Consequence %in% c("splice_acceptor_variant","splice_donor_variant","stop_gained","stop_lost","frameshift_variant"))))


regulatoryVariant<-VcfVepRules(exprs = list(RegVar = expression(Consequence %in% c("5_prime_UTR_variant", "3_prime_UTR_variant", "intron_variant","NMD_transcript_variant","upstream_gene_variant", "downstream_gene_variant"))))


combinedFilters <- VcfFilterRules(
  vepDistFilter,
  vepMAFFilter,
  vepCADDFilter, 
  highImpactVariant,
  regulatoryVariant)



You can play and add or remove filters

In [0]:
active(combinedFilters)


Distance       MAF      CADD BigImpact    RegVar 
     TRUE      TRUE      TRUE      TRUE      TRUE

Here we are gonna ficus only on regulatory variants

In [0]:
active(combinedFilters)["BigImpact"] <- FALSE

active(combinedFilters)["Distance"] <- FALSE

active(combinedFilters)["CADD"] <- FALSE

active(combinedFilters)

Distance       MAF      CADD BigImpact    RegVar 
    FALSE      TRUE     FALSE     FALSE      TRUE

#### Apply set of filters

In [0]:
vcf_filt <- subsetByFilter(vcf_het_cand, combinedFilters)
#rowRanges(vcf_filt)
csq_filt <- ensemblVEP::parseCSQToGRanges(x = vcf_filt)
csq_filt %>% filter(BIOTYPE=="protein_coding")
#vcf_filt

ERROR: ignored

In [0]:
summary(evalSeparately(expr = combinedFilters, envir = vcf_het_cand))
#as.data.frame(geno(vcf_rng)) %>% mutate(rsid=rownames(as.data.frame(geno(vcf_rng))))%>% filter(value %in% c("0|1", "1|1"))


#csq

<initial>  Distance       MAF      CADD BigImpact    RegVar   <final> 
        2         2         1         2         2         2         1